## Turning the rdf files into csv files

In [ ]:
#importing the required libraries
import pandas as pd
import os
import glob

#setting the working directory
data = r"C:/Users/hp/Desktop/monday_chall/Data"
rdf_path = r"C:/Users/hp/Desktop/monday_chall/Data/rdf_CTA__Ridership__Daily_by_Route_routes_2001_2025"

#reading the files
rdf_files = glob.glob(os.path.join(rdf_path, "*.rdf"))


**turning the rdf files into csv files**

In [ ]:
import xml.etree.ElementTree as ET
from tqdm import tqdm

# Define namespaces used in the RDF files
namespaces = {
    'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
    'ds': 'https://data.cityofchicago.org/resource/jyb9-n7fm/',
    'socrata': 'http://www.socrata.com/rdf/terms#',
    'rdfs': 'http://www.w3.org/2000/01/rdf-schema#'
}

def parse_rdf_file(file_path):
    """Parse a single RDF file and extract ridership data."""
    records = []
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        
        # Find all Description elements (each represents one record)
        for desc in root.findall('rdf:Description', namespaces):
            record = {}
            
            # Extract route
            route = desc.find('ds:route', namespaces)
            record['route'] = route.text if route is not None else None
            
            # Extract date
            date = desc.find('ds:date', namespaces)
            record['date'] = date.text if date is not None else None
            
            # Extract daytype
            daytype = desc.find('ds:daytype', namespaces)
            record['daytype'] = daytype.text if daytype is not None else None
            
            # Extract rides
            rides = desc.find('ds:rides', namespaces)
            record['rides'] = int(rides.text) if rides is not None else None
            
            records.append(record)
    except Exception as e:
        print(f"Error parsing {file_path}: {e}")
    
    return records

# Parse all RDF files and combine into a single list
all_records = []

print(f"Found {len(rdf_files)} RDF files to process")

for rdf_file in tqdm(rdf_files, desc="Processing RDF files"):
    records = parse_rdf_file(rdf_file)
    all_records.extend(records)
    
print(f"Total records extracted: {len(all_records)}")

# Create a DataFrame from all records
df = pd.DataFrame(all_records)

# Clean up the date column (remove the time portion)
df['date'] = pd.to_datetime(df['date']).dt.date

# Display
print("\nSample data:")
display(df.head())
print(f"\nDataFrame shape: {df.shape}")

# to CSV
output_file = os.path.join(data, "CTA_Ridership_Daily_by_Route_2001_2025.csv")
df.to_csv(output_file, index=False)
print(f"\nData saved to: {output_file}")

Found 19 RDF files to process


Processing RDF files: 100%|██████████| 19/19 [00:28<00:00,  1.50s/it]


Total records extracted: 1092474

Sample data:


,route,date,daytype,rides
0,1,2004-06-16,W,3174
1,100,2021-06-29,W,176
2,100,2018-05-21,W,497
3,106,2009-02-25,W,3146
4,100,2021-12-30,W,204



DataFrame shape: (1092474, 4)

Data saved to: C:/Users/hp/Desktop/monday_chall/Data\CTA_Ridership_Daily_by_Route_2001_2025.csv


now we have the files let's see what we have

In [5]:
#for philadelphia
phila_Mode = pd.read_csv(r"C:\Users\hp\Desktop\monday_chall\Data\Average_Daily_Ridership_By_Mode - City of Philadelphia.csv")
phila_Routes = pd.read_csv(r"C:\Users\hp\Desktop\monday_chall\Data\Average_Daily_Ridership_By_Route - City of Philadelphia.csv")
#for chicago
chicago_Mode = pd.read_excel(r"C:\Users\hp\Desktop\monday_chall\Data\cta-ridership-daily-boarding-totals-20260203-69820a3f9df63091665572.xlsx")
chicago_Routes = pd.read_csv(r"C:\Users\hp\Desktop\monday_chall\Data\CTA_Ridership_Daily_by_Route_2001_2025.csv")

C:\Users\hp\AppData\Local\Temp\ipykernel_23676\3395836435.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  chicago_Routes = pd.read_csv(r"C:\Users\hp\Desktop\monday_chall\Data\CTA_Ridership_Daily_by_Route_2001_2025.csv")


In [11]:
# Displaying the head of all 4 dataframes
print("=" * 50)
print("Philadelphia Mode Ridership")
display(phila_Mode.head())

print("\n" + "=" * 50)
print("Chicago Mode Ridership")

display(chicago_Mode.head())

print("\n" + "=" * 50)
print("Chicago Routes Ridership")

display(chicago_Routes.head())

print("\n" + "=" * 50)
print("Philadelphia Routes Ridership")

display(phila_Routes.head())


Philadelphia Mode Ridership


,Calendar_Year,Calendar_Month,Mode,Average_Daily_Ridership,Source,ObjectId
0,2019,1,Bus,459160,APC,1
1,2019,1,CCT,4294,Revenue,2
2,2019,1,Heavy Rail,296709,Revenue,3
3,2019,1,Regional Rail,122856,Revenue,4
4,2019,1,Trackless Trolley,16853,APC,5



Chicago Mode Ridership


,service_date,day_type,bus,rail_boardings,total_rides
0,2001-01-01,U,297192,126455,423647
1,2001-01-02,W,780827,501952,1282779
2,2001-01-03,W,824923,536432,1361355
3,2001-01-04,W,870021,550011,1420032
4,2001-01-05,W,890426,557917,1448343



Chicago Routes Ridership


,route,date,daytype,rides,year,month
0,1,2004-06-16,W,3174,2004,6
1,100,2021-06-29,W,176,2021,6
2,100,2018-05-21,W,497,2018,5
3,106,2009-02-25,W,3146,2009,2
4,100,2021-12-30,W,204,2021,12



Philadelphia Routes Ridership


,Calendar_Year,Calendar_Month,Route,Average_Daily_Ridership,Source,ObjectId
0,2019,8,59,3811,APC,2
1,2019,8,60,8550,APC,3
2,2019,8,61,3563,APC,4
3,2019,8,62,226,APC,5
4,2019,8,64,4335,APC,6


In [10]:
# Convert date column to datetime
chicago_Routes['date'] = pd.to_datetime(chicago_Routes['date'])

# Extract year and month into separate columns
chicago_Routes['year'] = chicago_Routes['date'].dt.year
chicago_Routes['month'] = chicago_Routes['date'].dt.month

display(chicago_Routes.head())

,route,date,daytype,rides,year,month
0,1,2004-06-16,W,3174,2004,6
1,100,2021-06-29,W,176,2021,6
2,100,2018-05-21,W,497,2018,5
3,106,2009-02-25,W,3146,2009,2
4,100,2021-12-30,W,204,2021,12


In [ ]:
# Convert date column to datetime
# Note: The column might be named 'service_date' in the Excel file - adjust if needed
chicago_Mode['service_date'] = pd.to_datetime(chicago_Mode['service_date'])

# Extract year and month into separate columns
chicago_Mode['year'] = chicago_Mode['service_date'].dt.year
chicago_Mode['month'] = chicago_Mode['service_date'].dt.month

display(chicago_Mode.head())

,service_date,day_type,bus,rail_boardings,total_rides,year,month
0,2001-01-01,U,297192,126455,423647,2001,1
1,2001-01-02,W,780827,501952,1282779,2001,1
2,2001-01-03,W,824923,536432,1361355,2001,1
3,2001-01-04,W,870021,550011,1420032,2001,1
4,2001-01-05,W,890426,557917,1448343,2001,1


In [13]:
# Displaying the head of all 4 dataframes
print("=" * 50)
print("Philadelphia Mode Ridership")
display(phila_Mode.head())

print("\n" + "=" * 50)
print("Chicago Mode Ridership")

display(chicago_Mode.head())

print("\n" + "=" * 50)
print("Chicago Routes Ridership")

display(chicago_Routes.head())

print("\n" + "=" * 50)
print("Philadelphia Routes Ridership")

display(phila_Routes.head())

Philadelphia Mode Ridership


,Calendar_Year,Calendar_Month,Mode,Average_Daily_Ridership,Source,ObjectId
0,2019,1,Bus,459160,APC,1
1,2019,1,CCT,4294,Revenue,2
2,2019,1,Heavy Rail,296709,Revenue,3
3,2019,1,Regional Rail,122856,Revenue,4
4,2019,1,Trackless Trolley,16853,APC,5



Chicago Mode Ridership


,service_date,day_type,bus,rail_boardings,total_rides,year,month
0,2001-01-01,U,297192,126455,423647,2001,1
1,2001-01-02,W,780827,501952,1282779,2001,1
2,2001-01-03,W,824923,536432,1361355,2001,1
3,2001-01-04,W,870021,550011,1420032,2001,1
4,2001-01-05,W,890426,557917,1448343,2001,1



Chicago Routes Ridership


,route,date,daytype,rides,year,month
0,1,2004-06-16,W,3174,2004,6
1,100,2021-06-29,W,176,2021,6
2,100,2018-05-21,W,497,2018,5
3,106,2009-02-25,W,3146,2009,2
4,100,2021-12-30,W,204,2021,12



Philadelphia Routes Ridership


,Calendar_Year,Calendar_Month,Route,Average_Daily_Ridership,Source,ObjectId
0,2019,8,59,3811,APC,2
1,2019,8,60,8550,APC,3
2,2019,8,61,3563,APC,4
3,2019,8,62,226,APC,5
4,2019,8,64,4335,APC,6


In [14]:
# Save chicago_Routes with the new year and month columns
chicago_Routes.to_csv(r"C:\Users\hp\Desktop\monday_chall\Data\CTA_Ridership_Daily_by_Route_2001_2025.csv", index=False)
print("Done 1")

# Save chicago_Mode with the new year and month columns
chicago_Mode.to_excel(r"C:\Users\hp\Desktop\monday_chall\Data\cta-ridership-daily-boarding-totals-20260203-69820a3f9df63091665572.xlsx", index=False)
print("Done 2")

Done 1
Done 2
